# This notebook is for use on cloud services where interactive use of matplotlib is hard. A script similar to this notebook is in in: snpe/marketplace_simulator_analysis.py

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Just a formatting related plugin
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt

<IPython.core.display.Javascript object>

In [4]:
import sys

sys.path.append("../")

<IPython.core.display.Javascript object>

In [5]:
import multiprocessing as mp

from collections import deque
from pathlib import Path
from typing import Dict, Optional

import arviz

import numpy as np
import pandas as pd
import pyreadr
import sbi
import sbi.utils as sbi_utils
import seaborn as sns
import statsmodels.formula.api as smf
import torch

from joblib import Parallel, delayed
from matplotlib.lines import Line2D
from scipy.stats import ttest_ind
from snpe.inference import inference_class
from snpe.simulations import simulator_class
from snpe.utils.statistics import review_histogram_correlation
from snpe.utils.tqdm_utils import tqdm_joblib
from tqdm import tqdm

# Set plotting parameters
sns.set(style="white", context="talk", font_scale=2.5)
sns.set_color_codes(palette="colorblind")
sns.set_style("ticks", {"axes.linewidth": 2.0})

<IPython.core.display.Javascript object>

In [6]:
ARTIFACT_PATH = Path("../../../gcs_mount/artifacts/marketplace")
# ARTIFACT_PATH = Path("/data/reputation-systems/snpe/artifacts/marketplace")

<IPython.core.display.Javascript object>

# Functions

In [7]:
def generate_and_save_simulations(
    num_simulations: int,
    review_prior: np.array,
    tendency_to_rate: float,
    simulation_type: str,
    previous_rating_measure: str,
    min_reviews_for_herding: int,
    herding_differentiating_measure: str,
) -> None:
    params = {
        "review_prior": review_prior,
        "tendency_to_rate": tendency_to_rate,
        "simulation_type": simulation_type,
        "previous_rating_measure": previous_rating_measure,
        "min_reviews_for_herding": min_reviews_for_herding,
        "herding_differentiating_measure": herding_differentiating_measure,
    }
    simulator = simulator_class.DoubleHerdingSimulator(params)
    simulator.simulate(num_simulations=num_simulations)
    simulator.save_simulations(ARTIFACT_PATH)

<IPython.core.display.Javascript object>

In [8]:
def infer_and_save_posterior(
    device: str, simulator_type: str, simulation_type: str, params: Dict
) -> None:
    parameter_prior = sbi_utils.BoxUniform(
        low=torch.tensor([0.0, 0.0, 0.0]).type(torch.FloatTensor),
        high=torch.tensor([4.0, 4.0, 1.0]).type(torch.FloatTensor),
        device=device,
    )
    inferrer = inference_class.TimeSeriesInference(
        parameter_prior=parameter_prior, device=device
    )
    inferrer.load_simulator(
        dirname=ARTIFACT_PATH,
        simulator_type=simulator_type,
        simulation_type=simulation_type,
    )
    batch_size = params.pop("batch_size")
    learning_rate = params.pop("learning_rate")
    hidden_features = params.pop("hidden_features")
    num_transforms = params.pop("num_transforms")
    inferrer.infer_snpe_posterior(
        embedding_net_conf=params,
        batch_size=batch_size,
        learning_rate=learning_rate,
        hidden_features=hidden_features,
        num_transforms=num_transforms,
    )
    inferrer.save_inference(ARTIFACT_PATH)

<IPython.core.display.Javascript object>

In [21]:
def sample_posterior_with_observed(
    device: str,
    observations: np.array,
    num_samples: int,
    simulator_type: str,
    simulation_type: str,
) -> np.array:
    # The parameter prior doesn't matter here as it will be overridden by that of the loaded inference object
    parameter_prior = sbi.utils.BoxUniform(
        low=torch.tensor([0.0, 0.0, 0.0]).type(torch.FloatTensor),
        high=torch.tensor([4.0, 4.0, 1.0]).type(torch.FloatTensor),
        device=device,
    )
    inferrer = inference_class.TimeSeriesInference(
        parameter_prior=parameter_prior, device=device
    )
    inferrer.load_simulator(
        dirname=ARTIFACT_PATH,
        simulator_type=simulator_type,
        simulation_type=simulation_type,
    )
    inferrer.load_inference(dirname=ARTIFACT_PATH)
    posterior_samples = inferrer.get_posterior_samples(
        observations, num_samples=num_samples
    )
    return posterior_samples

<IPython.core.display.Javascript object>

# Actual code to run the functions above

In [8]:
torch.set_num_threads(mp.cpu_count())
torch.get_num_threads()

16

<IPython.core.display.Javascript object>

In [16]:
inference_params = {
    "batch_size": 128,
    "learning_rate": 1.9e-4,
    "hidden_features": 35,
    "num_transforms": 5,
    "num_conv_layers": 2,
    "num_channels": 10,
    "conv_kernel_size": 5,
    "maxpool_kernel_size": 9,
    "num_dense_layers": 1,
}

<IPython.core.display.Javascript object>

In [17]:
infer_and_save_posterior("cuda", "marketplace", "timeseries", inference_params)

Embedding net created: 
 Sequential(
  (0): Conv1d(5, 10, kernel_size=(5,), stride=(1,), padding=(2,))
  (1): LeakyReLU(negative_slope=0.01)
  (2): Conv1d(10, 10, kernel_size=(5,), stride=(1,), padding=(4,), dilation=(2,))
  (3): MaxPool1d(kernel_size=9, stride=9, padding=0, dilation=1, ceil_mode=False)
  (4): Flatten(start_dim=1, end_dim=-1)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=740, out_features=32, bias=True)
)


/opt/conda/lib/python3.7/site-packages/sbi/utils/user_input_checks.py:684: UserWarning: Data x has device 'cpu' different from the training_device 'cuda:0', moving x to the training_device 'cuda:0'.
  f"Data x has device '{x.device}' "
/opt/conda/lib/python3.7/site-packages/sbi/utils/user_input_checks.py:692: UserWarning: Parameters theta has device 'cpu' different from the training_device 'cuda:0', moving theta to the training_device 'cuda:0'.
  f"Parameters theta has device '{theta.device}' "
/opt/conda/lib/python3.7/site-packages/sbi/utils/user_input_checks.py:428: UserWarning: Mismatch between the device of the data fed to the embedding_net and the device of the embedding_net's weights. Fed data has device 'cuda:0' vs embedding_net weights have device 'cpu'. Automatically switching the embedding_net's device to 'cuda:0', which could otherwise be done manually using the line `embedding_net.to('cuda:0')`.
  "Mismatch between the device of the data fed "


 Neural network successfully converged after 234 epochs.
        -------------------------
        ||||| ROUND 1 STATS |||||:
        -------------------------
        Epochs trained: 234
        Best validation performance: 1.0422
        -------------------------
        


<IPython.core.display.Javascript object>

# Working with the observed data

In [11]:
# reviews = pyreadr.read_r(
#    "/data/Dropbox/SNPE/all_data/reviews_bazaarvoice_main_vars.Rds"
# )
reviews = pyreadr.read_r(ARTIFACT_PATH / "reviews_bazaarvoice_main_vars.Rds")
reviews = reviews[None]
reviews.head()

,asin,overall,ReviewId,unixReviewTime,rootcategoryid
0,670155,4,3480905,2009-02-10,33005908
1,670155,3,3535864,2009-02-24,33005908
2,670155,5,3539112,2009-02-24,33005908
3,670155,4,3539514,2009-02-25,33005908
4,670155,1,3556157,2009-03-05,33005908


<IPython.core.display.Javascript object>

In [12]:
pd.testing.assert_frame_equal(reviews, reviews.sort_values(["asin", "unixReviewTime"]))

<IPython.core.display.Javascript object>

In [13]:
reviews.asin.value_counts().max()

1293

<IPython.core.display.Javascript object>

In [29]:
reviews.asin.unique().shape

(1406,)

<IPython.core.display.Javascript object>

In [30]:
# Pull out the ratings from the timeseries DF and convert them into a format
# that can be fed into the inference engine
timeseries_data = []
for product in reviews.asin.unique():
    # We were cutting off at 5001 reviews earlier when we could control the number of reviews per product in the sims
    # But now, we put the max len of the deque a little higher than the number of reviews
    # for the most reviewed product (around 1300)
    timeseries = deque([np.zeros(5)], maxlen=673)
    ratings = np.array(reviews.loc[reviews.asin == product, "overall"])
    ratings = ratings[:673]
    for rating in ratings:
        current_histogram = timeseries[-1].copy()
        current_histogram[int(rating - 1)] += 1
        timeseries.append(current_histogram)
    timeseries_data.append(np.array(timeseries))
timeseries_data = np.array(timeseries_data, dtype="object")

<IPython.core.display.Javascript object>

In [31]:
timeseries_data.shape

(1406,)

<IPython.core.display.Javascript object>

In [32]:
posterior_samples = sample_posterior_with_observed(
    "cuda", timeseries_data, 10_000, "marketplace", "timeseries"
)

/opt/conda/lib/python3.7/site-packages/sbi/utils/torchutils.py:28: UserWarning: GPU was selected as a device for training the neural network. Note that we expect **no** significant speed ups in training for the default architectures we provide. Using the GPU will be effective only for large neural networks with operations that are fast on the GPU, e.g., for a CNN or RNN `embedding_net`.
  "GPU was selected as a device for training the neural network. "


<IPython.core.display.Javascript object>

In [33]:
posterior_samples.shape

(10000, 1406, 3)

<IPython.core.display.Javascript object>

In [35]:
np.save(ARTIFACT_PATH / "posterior_samples_bazaarvoice.npy", posterior_samples)

<IPython.core.display.Javascript object>